# Fraud Detection using ClickHouse

In [1]:
!sudo apt-get install apt-transport-https ca-certificates dirmngr
!sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv E0C56BD4
!echo "deb http://repo.clickhouse.tech/deb/stable/ main/" | sudo tee \
    /etc/apt/sources.list.d/clickhouse.list
!sudo apt-get update
!sudo apt-get install -y clickhouse-server clickhouse-client
!sudo service clickhouse-server start
!clickhouse-client

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ca-certificates is already the newest version (20211016~18.04.1).
dirmngr is already the newest version (2.2.4-1ubuntu1.6).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 4,348 B of archives.
After this operation, 154 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.14 [4,348 B]
Fetched 4,348 B in 0s (31.9 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debco

In [2]:
!pip install clickhouse-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 622 kB 5.2 MB/s 


In [3]:
from clickhouse_driver import Client
import pandas as pd
from datetime import datetime

In [4]:
client = Client(host='localhost', password='fraud')

In [5]:
client.execute('CREATE DATABASE fraud_detection')
client.execute('SHOW DATABASES')

[('INFORMATION_SCHEMA',),
 ('default',),
 ('fraud_detection',),
 ('information_schema',),
 ('system',)]

In [6]:
client = Client(host='localhost',
                user='default',
                password='fraud',
                port=9000,
                database='fraud_detection',
                settings={'columnar': True}
         )

In [7]:
df = pd.read_csv('./transactions.csv')
df['dt'] = pd.to_datetime(df['dt'], format='%Y-%m-%d %H:%M:%S')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   sender  75 non-null     object        
 1   dt      75 non-null     datetime64[ns]
 2   amount  75 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.9+ KB


In [8]:
df.head()

,sender,dt,amount
0,GaiHxoxzAI,2022-04-07 05:57:04,35.39
1,GaiHxoxzAI,2022-04-07 05:12:46,60.78
2,GaiHxoxzAI,2022-04-07 21:30:06,96.28
3,GaiHxoxzAI,2022-04-07 01:51:59,62.29
4,GaiHxoxzAI,2022-04-07 09:49:55,57.10


In [9]:
client.execute('DROP TABLE IF EXISTS transactions')
client.execute(
    '''
    CREATE TABLE transactions (
        sender String,
        dt DateTime,
        amount Float32
    )
    ENGINE = Memory
    '''
)
client.execute('SHOW TABLES FROM fraud_detection')

[('transactions',)]

In [10]:
client.execute('INSERT INTO transactions VALUES', df.to_dict('records'))

75

In [11]:
client.execute('DESCRIBE transactions')

[('sender', 'String', '', '', '', '', ''),
 ('dt', 'DateTime', '', '', '', '', ''),
 ('amount', 'Float32', '', '', '', '', '')]

In [12]:
def select(query):
    return client.query_dataframe(query)

In [13]:
select(
    '''
    SELECT *
    FROM transactions
    LIMIT 10
    '''
)

,sender,dt,amount
0,GaiHxoxzAI,2022-04-07 05:57:04,35.389999
1,GaiHxoxzAI,2022-04-07 05:12:46,60.779999
2,GaiHxoxzAI,2022-04-07 21:30:06,96.279999
3,GaiHxoxzAI,2022-04-07 01:51:59,62.290001
4,GaiHxoxzAI,2022-04-07 09:49:55,57.099998
5,GaiHxoxzAI,2022-04-07 02:07:34,24.170000
6,GaiHxoxzAI,2022-04-07 23:51:43,85.449997
7,GaiHxoxzAI,2022-04-07 02:32:33,26.049999
8,RwahcKRENT,2022-04-07 01:26:02,25.780001
9,RwahcKRENT,2022-04-07 04:21:47,75.430000


The transaction chain is considered fraudulent if the following conditions are met:  
- each transaction from the same sender
- transactions' sum >= 150
- 2 or more consecutive transactions with interruption < 1 hour  
  
Print for each sender:  
Transaction chain start time, transaction chain end time, number of transactions in chain, sum of chain transactions

P.S. This task conditions is taken from HackerRank "Advanced SQL" certification. The data set for the solution was generated by me

In [14]:
select(
    '''
    WITH
    transactions_lags AS (
        SELECT sender,
               dt,
               lagInFrame(dt, 1) OVER (
                   PARTITION BY sender
                   ORDER BY dt
                   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS time_lag,
               amount
        FROM transactions),
    transactions_hours AS (
        SELECT sender,
               dt,
               dateDiff('minute', time_lag, dt) AS minute_diff,
               amount
        FROM transactions_lags),
    transactions_series AS (
        SELECT sender,
               dt,
               COUNT(IF(minute_diff > 60, 1, NULL)) OVER (
                   PARTITION BY sender
                   ORDER BY dt) AS series,
               amount
        FROM transactions_hours),
    transactions_all AS (
        SELECT sender,
               FIRST_VALUE(dt) OVER (
                   PARTITION BY sender, series
                   ORDER BY dt) AS transactions_start,
               LAST_VALUE(dt) OVER (
                   PARTITION BY sender, series
                   ORDER BY dt
                   ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS transactions_end,
               ROW_NUMBER() OVER (
                   PARTITION BY sender, series
                   ORDER BY dt) AS trans_num_in_series,
               SUM(amount) OVER (
               PARTITION BY sender, series
               ORDER BY dt) AS transactions_sum
        FROM transactions_series)
    SELECT sender,
           transactions_start,
           transactions_end,
           MAX(trans_num_in_series) AS transactions_count,
           ROUND(MAX(transactions_sum), 2) AS transactions_sum
    FROM transactions_all
    GROUP BY sender, transactions_start, transactions_end
    HAVING transactions_count > 1 AND transactions_sum >= 150
    ORDER BY sender
    '''
)

,sender,transactions_start,transactions_end,transactions_count,transactions_sum
0,PEcbzs4jzR,2022-04-07 11:19:30,2022-04-07 11:35:53,2,193.92
1,PEcbzs4jzR,2022-04-07 04:06:48,2022-04-07 05:21:01,3,206.81
2,g7ivjANgTO,2022-04-07 08:33:28,2022-04-07 09:04:44,4,207.30


As we can see we can easily use ClickHouse for this task, as well as MySQL or PostgreSQL. The advantage of using ClickHouse is that it allows to solve this task in real time.